# Calculating mutual information by modeling P(R|S) as gaussian

The mutual information of the response R and stimulus S can be expressed by

$I(R;S) = \sum\limits_{s\in S} \int\limits_{R} dr \, p(r, s) \log_2{\frac{p(r, s)}{p(r)p(s)}}$

If we model $P(R|S)$ as gaussian by fitting the mean and covariance to points with the same stimulus label, then $P(R)$ is a mixture of gaussians model expressed as $P(R) = P(R|S)P(S)$

We can compute a monte carlo estimate of $I(R;S)$ by sampling from the distribution $P(R,S)$. A sampled point $(r_i, s_i)$ can be selected by first picking an $s_i$ from $S$ (weighted by frequency of presentation of $s_i$), and then sampling an $r_i$ from the multivariate gaussian $p(r|s_i)$.

The monte carlo estimate of $I(R;S)$ over the $N$ sampled points $(r_i, s_i)$ is 

$\tilde{I}(R;S) = \frac{1}{N} \sum\limits_{i=1}^N \log_2{\frac{p(r_i | s_i)}{p(r_i)}}$

with

$p(r_i | s_i) = \mathrm{NormPDF}(\mu_{s_i}, \Sigma_{s_i}, r_i)$

$p(r_i) = \sum\limits_{s_i \in S} p(r_i | s_i)  p(s_i) $

In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt
import sklearn.decomposition

import config
from load import DataExplorer
from mutual_information import monte_carlo_mutual_information
from process_spikes import bin_spikes, conv, exponential_convolver
from fit_gaussians import fit_gaussians

/Users/kevinyu/Projects/mds-bird/env27/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
BIRD = "GreBlu9508M"
SITE = 2
UNIT = (21, 1)

In [4]:
dat = DataExplorer(
    config.DATADIR,
    BIRD,
    SITE,
    exclude_noise=True,
    exclude_song=True)

In [5]:
table, spike_times = dat.load_table(filter_unit=UNIT, load_spike_times=True)
_, spikes = bin_spikes(spike_times, min_time=config.MIN_TIME, max_time=config.MAX_TIME)
psths = conv(spikes, exponential_convolver, config.WIDTH)

### By Stimulus Identity

First, we can estimate the mutual information by dividing our responses up by stimulus identity.

In [6]:
pca = sklearn.decomposition.PCA(n_components=2)
x = pca.fit_transform(psths)

dists, p = fit_gaussians(table, x, key="stim")
    
mutual_info, err = monte_carlo_mutual_information(dists, n=200)
print "{:.2f} ({:.2f}) / {:.2f} bits".format(
    mutual_info,
    err,
    np.log2(len(np.unique(table["stim"]))))

1.74 (0.11) / 6.54 bits


### By Stimulus Category

Next, we can estimate the mutual information when dividing our responses up by stimulus category (i.e. aggressive, distance, etc)  

In [7]:
pca = sklearn.decomposition.PCA(n_components=10)
x = pca.fit_transform(psths)

dists, p = fit_gaussians(table, x, key="stim_type")
    
mutual_info, err = monte_carlo_mutual_information(dists, n=200)
print "{:.2f} ({:.2f}) / {:.2f} bits".format(
    mutual_info,
    err,
    np.log2(len(np.unique(table["stim_type"]))))

2.64 (0.06) / 3.00 bits
